In [12]:
import csv
import os

# ==========================================
# CONFIGURATION
# ==========================================
OUTPUT_FILENAME = "beers_master_template.csv"

# The Definitive Schema Headers
HEADERS = [
    "source_table", 
    "category_or_disease", 
    "drug_name", 
    "condition_detail", 
    "rationale", 
    "recommendation", 
    "quality", 
    "strength"
]

# Example rows to guide your manual entry
EXAMPLE_DATA = [
    {
        "source_table": "Table 2 (PIM)",
        "category_or_disease": "Antihistamines",
        "drug_name": "Diphenhydramine",
        "condition_detail": "N/A",
        "rationale": "Highly anticholinergic; risk of confusion, dry mouth, constipation.",
        "recommendation": "Avoid",
        "quality": "Moderate",
        "strength": "Strong"
    },
    {
        "source_table": "Table 3 (Disease)",
        "category_or_disease": "Heart Failure",
        "drug_name": "Diltiazem",
        "condition_detail": "N/A",
        "rationale": "Potential to promote fluid retention and exacerbate heart failure.",
        "recommendation": "Avoid in HFrEF",
        "quality": "Moderate",
        "strength": "Strong"
    },
    {
        "source_table": "Table 5 (Interactions)",
        "category_or_disease": "Drug-Interaction",
        "drug_name": "Warfarin",
        "condition_detail": "Amiodarone",
        "rationale": "Increased risk of bleeding.",
        "recommendation": "Avoid when possible; monitor INR closely.",
        "quality": "Moderate",
        "strength": "Strong"
    },
    {
        "source_table": "Table 6 (Renal)",
        "category_or_disease": "Anti-infective",
        "drug_name": "Ciprofloxacin",
        "condition_detail": "CrCl <30 mL/min",
        "rationale": "Increased risk of CNS effects (seizures) and tendon rupture.",
        "recommendation": "Reduce dose",
        "quality": "Moderate",
        "strength": "Strong"
    }
]

def create_template_csv():
    print(f"Creating {OUTPUT_FILENAME}...")
    
    try:
        with open(OUTPUT_FILENAME, mode='w', newline='', encoding='utf-8') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=HEADERS)
            
            # Write the header row
            writer.writeheader()
            
            # Write the example rows
            writer.writerows(EXAMPLE_DATA)
            
        print(f"Success! File '{OUTPUT_FILENAME}' created.")
        print("You can now open this file in Excel and start pasting your data.")
        
    except IOError as e:
        print(f"Error writing to file: {e}")

if __name__ == "__main__":
    create_template_csv()

Creating beers_master_template.csv...
Success! File 'beers_master_template.csv' created.
You can now open this file in Excel and start pasting your data.


In [13]:
import pdfplumber
import pandas as pd
import re

# ==========================================
# CONFIGURATION
# ==========================================
PDF_PATH = "STOPP-START-V3.pdf"
OUTPUT_FILE = "stopp_start_v3_rules.csv"

# Regex to find the start of a rule (e.g., "1. Digoxin...")
RULE_PATTERN = re.compile(r'^(\d+)\.\s+(.*)')

# Regex to find Section Headers (e.g., "Section B: Cardiovascular System")
SECTION_PATTERN = re.compile(r'^Section\s+([A-Z]):\s*(.*)', re.IGNORECASE)

def clean_text(text):
    return re.sub(r'\s+', ' ', text).strip()

def parse_stopp_rule(text):
    """
    Parses a STOPP rule into Drug, Condition, Rationale.
    Logic: 
    1. Rationale is almost always in parentheses at the end.
    2. Drug is usually at the start.
    3. Condition is the text in between.
    """
    rationale = ""
    drug = ""
    condition = ""
    
    # 1. Extract Rationale (content in the last set of parentheses)
    # We look for the last (...) block
    paren_match = re.search(r'\(([^)]+)\)[^()]*$', text)
    if paren_match:
        rationale = paren_match.group(1).strip()
        # Remove the rationale from the main text to process the rest
        text_no_rat = text[:paren_match.start()].strip()
    else:
        text_no_rat = text
        rationale = "Review Manually"

    # 2. Split Drug vs Condition
    # Heuristic: Look for the first occurrence of prepositions
    splitters = [" with ", " in ", " for ", " where ", " unless "]
    split_idx = len(text_no_rat)
    
    for s in splitters:
        idx = text_no_rat.find(s)
        if idx != -1 and idx < split_idx:
            split_idx = idx
            
    if split_idx != len(text_no_rat):
        drug = text_no_rat[:split_idx].strip()
        condition = text_no_rat[split_idx:].strip() # Keep the preposition for context
    else:
        # Fallback
        drug = text_no_rat
        condition = "General/Unspecified"

    return drug, condition, rationale

def parse_start_rule(text):
    """
    Parses a START rule into Missing_Medication and Condition.
    Logic: "Drug X [for/in/with] Condition Y"
    """
    # START rules often don't have a rationale in parentheses, or the parens contain drug examples.
    # We essentially want to split the "Medication to Start" from the "Condition".
    
    medication = ""
    condition = ""
    
    # Heuristic: Look for the first occurrence of prepositions
    splitters = [" with ", " in ", " for ", " where ", " unless "]
    split_idx = len(text)
    
    for s in splitters:
        idx = text.find(s)
        if idx != -1 and idx < split_idx:
            split_idx = idx
            
    if split_idx != len(text):
        medication = text[:split_idx].strip()
        condition = text[split_idx:].strip()
    else:
        medication = text
        condition = "Review Manually"
        
    return medication, condition

def extract_stopp_start(pdf_path):
    print(f"Processing {pdf_path}...")
    
    records = []
    
    with pdfplumber.open(pdf_path) as pdf:
        current_tool = "STOPP" # Default start
        current_section = "General"
        current_rule_buffer = ""
        current_rule_id = ""
        
        for i, page in enumerate(pdf.pages):
            text = page.extract_text()
            if not text: continue
            
            lines = text.split('\n')
            
            for line in lines:
                clean_line = line.strip()
                
                # 1. Detect Switch to START criteria
                if "Screening Tool to Alert to Right Treatment" in clean_line or "START" in clean_line and "version 3" in clean_line:
                    # Flush previous buffer
                    if current_rule_buffer:
                        # Process previous rule before switching
                        records.append(process_buffer(current_tool, current_section, current_rule_id, current_rule_buffer))
                        current_rule_buffer = ""
                    
                    current_tool = "START"
                    current_section = "General" # Reset section
                    print(f"Switched to START criteria on Page {i+1}")
                    continue

                # 2. Detect Section Headers
                section_match = SECTION_PATTERN.match(clean_line)
                if section_match:
                    # Flush previous buffer
                    if current_rule_buffer:
                        records.append(process_buffer(current_tool, current_section, current_rule_id, current_rule_buffer))
                        current_rule_buffer = ""
                        
                    current_section = f"Section {section_match.group(1)}: {section_match.group(2)}"
                    continue

                # 3. Detect Numbered Rules (The start of a new entry)
                rule_match = RULE_PATTERN.match(clean_line)
                if rule_match:
                    # Flush previous buffer
                    if current_rule_buffer:
                        records.append(process_buffer(current_tool, current_section, current_rule_id, current_rule_buffer))
                    
                    # Start new buffer
                    current_rule_id = rule_match.group(1)
                    current_rule_buffer = rule_match.group(2)
                else:
                    # 4. Continuation line
                    # If we are inside a rule, append the text (it's likely a multi-line sentence)
                    if current_rule_buffer and clean_line and "Page" not in clean_line and "Appendix" not in clean_line:
                        current_rule_buffer += " " + clean_line

        # Flush final buffer
        if current_rule_buffer:
            records.append(process_buffer(current_tool, current_section, current_rule_id, current_rule_buffer))

    return pd.DataFrame(records)

def process_buffer(tool, section, r_id, text):
    """Helper to process the full text of a rule."""
    full_text = clean_text(text)
    
    if tool == "STOPP":
        drug, condition, rationale = parse_stopp_rule(full_text)
        return {
            "Type": "STOPP",
            "Section": section,
            "Rule_ID": r_id,
            "Drug_Medication": drug,
            "Condition_Disease": condition,
            "Rationale_Reason": rationale,
            "Full_Text": full_text
        }
    else:
        # START
        med, cond = parse_start_rule(full_text)
        return {
            "Type": "START",
            "Section": section,
            "Rule_ID": r_id,
            "Drug_Medication": med,
            "Condition_Disease": cond,
            "Rationale_Reason": "Omission of indicated therapy", # Default for START
            "Full_Text": full_text
        }

if __name__ == "__main__":
    df = extract_stopp_start(PDF_PATH)
    
    # Post-processing to clean up any "Page X" artifacts that slipped in
    df = df[~df["Full_Text"].str.contains("Appendix 1", na=False)]
    
    df.to_csv(OUTPUT_FILE, index=False, encoding='utf-8-sig')
    print(f"Success! Extracted {len(df)} rules to {OUTPUT_FILE}")
    print(df.head())

Processing STOPP-START-V3.pdf...
Switched to START criteria on Page 11
Success! Extracted 190 rules to stopp_start_v3_rules.csv
    Type                              Section Rule_ID  \
0  STOPP  Section A: Indication of medication       1   
1  STOPP  Section A: Indication of medication       2   
2  STOPP  Section A: Indication of medication       3   
3  STOPP     Section B: Cardiovascular System       1   
4  STOPP     Section B: Cardiovascular System       2   

                                     Drug_Medication  \
0  Any drug prescribed without a clinical indicat...   
1  Any drug prescribed beyond the recommended dur...   
2              Any duplicate drug class prescription   
3                                            Digoxin   
4                             Verapamil or diltiazem   

                                   Condition_Disease  \
0                                General/Unspecified   
1          where treatment duration is well defined.   
2  for daily regular use

In [14]:
import csv

OUTPUT_FILE = "tapering_rules_dataset.csv"

HEADERS = [
    "drug_class",
    "drug_name",
    "risk_profile",
    "taper_strategy_name",
    "step_logic",
    "switch_recommendation",
    "monitoring_frequency",
    "withdrawal_symptoms",
    "pause_criteria"
]

DATA = [
    # --- PPIs ---
    {
        "drug_class": "PPI",
        "drug_name": "Omeprazole",
        "risk_profile": "Standard",
        "taper_strategy_name": "Dose Reduction",
        "step_logic": "Reduce dose by 50% (e.g., 40mg to 20mg) daily for 4 weeks, then stop.",
        "switch_recommendation": "N/A",
        "monitoring_frequency": "At 4 and 12 weeks",
        "withdrawal_symptoms": "Rebound heartburn, dyspepsia (usually mild, lasts 3-7 days)",
        "pause_criteria": "Return of severe symptoms lasting >3 days interfering with sleep/activities."
    },
    {
        "drug_class": "PPI",
        "drug_name": "All PPIs",
        "risk_profile": "Standard",
        "taper_strategy_name": "Alternate Day",
        "step_logic": "Take standard dose every OTHER day for 4 weeks, then stop.",
        "switch_recommendation": "N/A",
        "monitoring_frequency": "At 4 and 12 weeks",
        "withdrawal_symptoms": "Rebound heartburn",
        "pause_criteria": "Return of severe symptoms."
    },
    
    # --- BZDs ---
    {
        "drug_class": "Benzodiazepine",
        "drug_name": "Alprazolam",
        "risk_profile": "High Risk (Short Half-life)",
        "taper_strategy_name": "Ashton Substitution",
        "step_logic": "Step 1: Replace 50% of dose with equivalent Diazepam. Step 2: Replace remaining 50%. Step 3: Taper Diazepam by 10% every 2 weeks.",
        "switch_recommendation": "Switch to Diazepam (1mg Alprazolam = 20mg Diazepam approx).",
        "monitoring_frequency": "Every 1-2 weeks",
        "withdrawal_symptoms": "Anxiety, tremors, insomnia, tachycardia, seizures (if abrupt).",
        "pause_criteria": "Unmanageable anxiety or physical tremors. Hold dose until stable (2-4 weeks)."
    },
    {
        "drug_class": "Benzodiazepine",
        "drug_name": "Clonazepam",
        "risk_profile": "Moderate",
        "taper_strategy_name": "Direct Taper",
        "step_logic": "Reduce Total Daily Dose by 5-10% every 2-4 weeks.",
        "switch_recommendation": "Optional: Switch to Diazepam if withdrawals are severe.",
        "monitoring_frequency": "Every 2 weeks",
        "withdrawal_symptoms": "Insomnia, anxiety, muscle tension.",
        "pause_criteria": "Severe withdrawal symptoms."
    },

    # --- SSRIs ---
    {
        "drug_class": "SSRI",
        "drug_name": "Paroxetine",
        "risk_profile": "High Risk (Severe Withdrawal)",
        "taper_strategy_name": "Hyperbolic Taper",
        "step_logic": "Reduce by 10% of current dose every 2-4 weeks. Go slower as dose gets lower (e.g. 20mg -> 18mg -> ... -> 5mg -> 2.5mg). Liquid formulation may be needed.",
        "switch_recommendation": "Consider switching to Fluoxetine (long half-life) if tapering Paroxetine is intolerable.",
        "monitoring_frequency": "Every 4 weeks",
        "withdrawal_symptoms": "Brain zaps, dizziness, nausea, lethargy, irritability (FINISH syndrome).",
        "pause_criteria": "Onset of 'brain zaps' or severe dizziness. Re-instate previous dose."
    },
    {
        "drug_class": "SSRI",
        "drug_name": "Fluoxetine",
        "risk_profile": "Low Risk (Long Half-life)",
        "taper_strategy_name": "Standard Taper",
        "step_logic": "Reduce dose by 50% every 4 weeks. Can often stop from 10mg or 20mg directly due to self-tapering effect.",
        "switch_recommendation": "N/A",
        "monitoring_frequency": "Every 4 weeks",
        "withdrawal_symptoms": "Minimal compared to others. Insomnia, mild anxiety.",
        "pause_criteria": "Severe relapse of depressive symptoms."
    },

    # --- Z-Drugs ---
    {
        "drug_class": "Z-Drug",
        "drug_name": "Zolpidem",
        "risk_profile": "Moderate",
        "taper_strategy_name": "Slow Taper",
        "step_logic": "Reduce dose by 25% every 2 weeks. (e.g. 10mg -> 7.5mg -> 5mg -> 2.5mg -> Stop).",
        "switch_recommendation": "Switch to Diazepam 5mg if direct taper fails.",
        "monitoring_frequency": "Every 2 weeks",
        "withdrawal_symptoms": "Rebound insomnia, anxiety.",
        "pause_criteria": "Total inability to sleep for >3 nights."
    }
]

def generate_csv():
    with open(OUTPUT_FILE, 'w', newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=HEADERS)
        writer.writeheader()
        writer.writerows(DATA)
    
    print(f"Successfully created '{OUTPUT_FILE}' with {len(DATA)} protocols.")

if __name__ == "__main__":
    generate_csv()

Successfully created 'tapering_rules_dataset.csv' with 7 protocols.


In [15]:
import csv

OUTPUT_FILE = "cfs_risk_taper_map.csv"

HEADERS = [
    "cfs_score",
    "clinical_label",
    "frailty_category",
    "risk_weight",          # Use this to increase monitoring frequency (e.g., Frequency / Risk Weight)
    "taper_speed_multiplier", # Multiply dose reduction step by this (e.g. 5mg * 0.5 = 2.5mg reduction)
    "clinical_guidance"
]

DATA = [
    {
        "cfs_score": 1,
        "clinical_label": "Very Fit",
        "frailty_category": "Fit",
        "risk_weight": 1.0,
        "taper_speed_multiplier": 1.0,
        "clinical_guidance": "Standard tapering protocols apply. Low risk of withdrawal complications."
    },
    {
        "cfs_score": 2,
        "clinical_label": "Well",
        "frailty_category": "Fit",
        "risk_weight": 1.0,
        "taper_speed_multiplier": 1.0,
        "clinical_guidance": "Standard tapering protocols apply."
    },
    {
        "cfs_score": 3,
        "clinical_label": "Managing Well",
        "frailty_category": "Fit",
        "risk_weight": 1.0,
        "taper_speed_multiplier": 1.0,
        "clinical_guidance": "Standard tapering protocols apply. Monitor for latent symptoms."
    },
    {
        "cfs_score": 4,
        "clinical_label": "Vulnerable",
        "frailty_category": "Vulnerable",
        "risk_weight": 1.25,
        "taper_speed_multiplier": 0.75,
        "clinical_guidance": "Consider extending taper duration by 25%. Monitor for 'unmasking' of symptoms."
    },
    {
        "cfs_score": 5,
        "clinical_label": "Mildly Frail",
        "frailty_category": "Frail",
        "risk_weight": 1.5,
        "taper_speed_multiplier": 0.5,
        "clinical_guidance": "Physiological reserve is reduced. Cut standard dose reduction steps in half (e.g., reduce by 10% instead of 20%)."
    },
    {
        "cfs_score": 6,
        "clinical_label": "Moderately Frail",
        "frailty_category": "Frail",
        "risk_weight": 1.75,
        "taper_speed_multiplier": 0.5,
        "clinical_guidance": "High risk of withdrawal causing confusion/falls. Use 'Hyperbolic' (very slow) tapering."
    },
    {
        "cfs_score": 7,
        "clinical_label": "Severely Frail",
        "frailty_category": "Severe Frailty",
        "risk_weight": 2.0,
        "taper_speed_multiplier": 0.25,
        "clinical_guidance": "Extreme Caution. Withdrawal stress may precipitate delirium. Extend taper steps to 6-8 weeks."
    },
    {
        "cfs_score": 8,
        "clinical_label": "Very Severely Frail",
        "frailty_category": "Severe Frailty",
        "risk_weight": 2.5,
        "taper_speed_multiplier": 0.25,
        "clinical_guidance": "Approaching end of life. Only deprescribe if medication is causing overt harm. Taper very slowly."
    },
    {
        "cfs_score": 9,
        "clinical_label": "Terminally Ill",
        "frailty_category": "Terminal",
        "risk_weight": 0.0,
        "taper_speed_multiplier": 0.0,
        "clinical_guidance": "Symptom control is priority. Deprescribing is based on 'Time to Benefit'. If drug provides no symptom relief, stop immediately. If withdrawal causes distress, maintain dose."
    }
]

def generate_csv():
    with open(OUTPUT_FILE, 'w', newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=HEADERS)
        writer.writeheader()
        writer.writerows(DATA)
    
    print(f"Successfully created '{OUTPUT_FILE}' with CFS mappings.")

if __name__ == "__main__":
    generate_csv()

Successfully created 'cfs_risk_taper_map.csv' with CFS mappings.


In [16]:
import csv

OUTPUT_FILE = "gender_risk_medications.csv"

HEADERS = [
    "risk_category",
    "drug_class",
    "drug_name",
    "gender_risk",
    "risk_level",
    "mechanism",
    "monitoring_guidance"
]

DATA = [
    # --- QT PROLONGATION (Female Risk) ---
    {
        "risk_category": "QT Prolongation",
        "drug_class": "Antiarrhythmic",
        "drug_name": "Sotalol",
        "gender_risk": "Female > Male",
        "risk_level": "High",
        "mechanism": "Estrogen-mediated reduction in repolarization reserve.",
        "monitoring_guidance": "Monitor ECG 2-4 hours after first dose. Avoid if QTc > 500ms."
    },
    {
        "risk_category": "QT Prolongation",
        "drug_class": "SSRI",
        "drug_name": "Citalopram",
        "gender_risk": "Female > Male",
        "risk_level": "Moderate",
        "mechanism": "Dose-dependent QTc prolongation.",
        "monitoring_guidance": "Max dose 20mg in older adults (>60 years). Annual ECG recommended."
    },
    {
        "risk_category": "QT Prolongation",
        "drug_class": "Antipsychotic",
        "drug_name": "Haloperidol",
        "gender_risk": "Female > Male",
        "risk_level": "High (IV), Moderate (Oral)",
        "mechanism": "Blockade of hERG potassium channels.",
        "monitoring_guidance": "Avoid IV use without cardiac monitoring. Check Mg/K levels."
    },
    {
        "risk_category": "QT Prolongation",
        "drug_class": "Antibiotic",
        "drug_name": "Erythromycin",
        "gender_risk": "Female > Male",
        "risk_level": "Moderate",
        "mechanism": "Direct channel blockade.",
        "monitoring_guidance": "Avoid in patients with baseline QTc > 450ms (men) or > 470ms (women)."
    },

    # --- HYPONATREMIA (Female Risk) ---
    {
        "risk_category": "Hyponatremia",
        "drug_class": "Thiazide Diuretic",
        "drug_name": "Hydrochlorothiazide",
        "gender_risk": "Female > Male",
        "risk_level": "High",
        "mechanism": "Impaired free water excretion and polydipsia. Older women account for >70% of severe cases.",
        "monitoring_guidance": "Check Sodium (Na) within 1-2 weeks of initiation. Stop if Na < 130 mmol/L."
    },
    {
        "risk_category": "Hyponatremia",
        "drug_class": "Antiepileptic",
        "drug_name": "Carbamazepine",
        "gender_risk": "Female > Male",
        "risk_level": "High",
        "mechanism": "Increases sensitivity to ADH (SIADH).",
        "monitoring_guidance": "Baseline Na and monthly monitoring for first 3 months."
    },
    {
        "risk_category": "Hyponatremia",
        "drug_class": "SSRI",
        "drug_name": "Sertraline",
        "gender_risk": "Female > Male",
        "risk_level": "Moderate",
        "mechanism": "SIADH (Syndrome of Inappropriate Antidiuretic Hormone).",
        "monitoring_guidance": "Check Na if patient presents with confusion, lethargy, or falls."
    },

    # --- FALL RISK / SEDATION (Pharmacokinetic Gender Diff) ---
    {
        "risk_category": "Sedation / Fall Risk",
        "drug_class": "Z-Drug (Hypnotic)",
        "drug_name": "Zolpidem",
        "gender_risk": "Female > Male",
        "risk_level": "High",
        "mechanism": "Lower clearance rates in women lead to higher blood concentrations the next morning.",
        "monitoring_guidance": "FDA Requirement: Max starting dose for women is 5mg (vs 10mg for men)."
    },
    {
        "risk_category": "Sedation / Fall Risk",
        "drug_class": "Benzodiazepine",
        "drug_name": "Diazepam",
        "gender_risk": "Female > Male",
        "risk_level": "High",
        "mechanism": "Lipophilic drug accumulates more in women (higher body fat %), extending half-life.",
        "monitoring_guidance": "Avoid long-acting agents. Use lower doses."
    }
]

def generate_csv():
    with open(OUTPUT_FILE, 'w', newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=HEADERS)
        writer.writeheader()
        writer.writerows(DATA)
    
    print(f"Successfully created '{OUTPUT_FILE}' with gender-based risks.")

if __name__ == "__main__":
    generate_csv()

Successfully created 'gender_risk_medications.csv' with gender-based risks.


In [17]:
import csv

OUTPUT_FILE = "time_to_benefit_dataset.csv"

HEADERS = [
    "drug_class",
    "drug_name",
    "indication_context",
    "time_to_benefit",      # String description
    "ttb_months_min",       # Integer for logic (lower bound)
    "ttb_months_max",       # Integer for logic (upper bound)
    "deprescribing_guidance",
    "reference_trial"
]

DATA = [
    # --- STATINS ---
    {
        "drug_class": "Statin",
        "drug_name": "All Statins",
        "indication_context": "Primary Prevention (No history of MI/Stroke)",
        "time_to_benefit": "2 to 5 Years",
        "ttb_months_min": 24,
        "ttb_months_max": 60,
        "deprescribing_guidance": "Deprescribe if Life Expectancy < 2-5 years. The harm (muscle pain, pill burden) likely outweighs the CV benefit.",
        "reference_trial": "PROSPER Trial / Statin Trialists"
    },
    {
        "drug_class": "Statin",
        "drug_name": "All Statins",
        "indication_context": "Secondary Prevention (History of MI/Stroke)",
        "time_to_benefit": "6 to 12 Months",
        "ttb_months_min": 6,
        "ttb_months_max": 12,
        "deprescribing_guidance": "Benefit is rapid. Maintain therapy unless patient is in end-of-life/palliative care or experiencing severe side effects.",
        "reference_trial": "4S Trial / LIPID"
    },

    # --- BONE HEALTH ---
    {
        "drug_class": "Bisphosphonate",
        "drug_name": "Alendronate, Risedronate",
        "indication_context": "Osteoporosis (Fracture Prevention)",
        "time_to_benefit": "8 to 19 Months",
        "ttb_months_min": 8,
        "ttb_months_max": 19,
        "deprescribing_guidance": "Stop if Life Expectancy < 1-2 years. Also consider 'Drug Holiday' after 5 years of use due to atypical fracture risk.",
        "reference_trial": "FIT Trial / HORIZON-PFT"
    },

    # --- DIABETES ---
    {
        "drug_class": "Antihyperglycemic",
        "drug_name": "Insulin, Sulfonylureas",
        "indication_context": "Strict Control (HbA1c < 7%) to prevent microvascular complications",
        "time_to_benefit": "8 Years",
        "ttb_months_min": 96,
        "ttb_months_max": 120,
        "deprescribing_guidance": "Strict control takes ~8 years to show benefit (UKPDS). If LE < 8 years, relax target to HbA1c 8.0-9.0% to avoid hypoglycemia.",
        "reference_trial": "UKPDS / ACCORD"
    },

    # --- HYPERTENSION ---
    {
        "drug_class": "Antihypertensive",
        "drug_name": "Thiazides, ACEI, CCB",
        "indication_context": "Stroke/CV Prevention",
        "time_to_benefit": "1 to 2 Years",
        "ttb_months_min": 12,
        "ttb_months_max": 24,
        "deprescribing_guidance": "Benefit is relatively fast. Continue unless hypotension or orthostasis causes falls. Deprescribe in severe frailty.",
        "reference_trial": "HYVET Trial"
    },

    # --- ASPIRIN ---
    {
        "drug_class": "Antiplatelet",
        "drug_name": "Aspirin",
        "indication_context": "Primary Prevention (Elderly)",
        "time_to_benefit": "None (Harm > Benefit)",
        "ttb_months_min": 999, # Sentinel value for "Never"
        "ttb_months_max": 999,
        "deprescribing_guidance": "Deprescribe immediately. ASPREE trial showed increased bleeding/mortality with NO cardiovascular benefit in healthy elderly >70.",
        "reference_trial": "ASPREE Trial (2018)"
    },

    # --- DEMENTIA ---
    {
        "drug_class": "Cholinesterase Inhibitor",
        "drug_name": "Donepezil, Rivastigmine",
        "indication_context": "Alzheimer's Dementia",
        "time_to_benefit": "12 Weeks (Symptomatic)",
        "ttb_months_min": 3,
        "ttb_months_max": 3,
        "deprescribing_guidance": "This is symptomatic only (not disease modifying). If no improvement seen in 3-6 months, or if dementia progresses to severe stage (unable to interact), stop.",
        "reference_trial": "Cochrane Reviews"
    },

    # --- SUPPLEMENTS ---
    {
        "drug_class": "Supplement",
        "drug_name": "Multivitamins, Fish Oil",
        "indication_context": "General Health / Prevention",
        "time_to_benefit": "None proven",
        "ttb_months_min": 999,
        "ttb_months_max": 999,
        "deprescribing_guidance": "Deprescribe to reduce pill burden. No evidence of mortality benefit in elderly.",
        "reference_trial": "USPSTF"
    }
]

def generate_csv():
    with open(OUTPUT_FILE, 'w', newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=HEADERS)
        writer.writeheader()
        writer.writerows(DATA)
    
    print(f"Successfully created '{OUTPUT_FILE}'.")

if __name__ == "__main__":
    generate_csv()

Successfully created 'time_to_benefit_dataset.csv'.


In [18]:
import pandas as pd
import json
from datetime import datetime

# ============================================================================
# AYURVEDIC KNOWN INTERACTIONS DATASET
# Based on MSK, NCCIH, PubMed systematic reviews, and validated clinical data
# ============================================================================

ayurvedic_known_interactions = {
    'herb_name': [],
    'scientific_name': [],
    'interacting_drug_class': [],
    'specific_drugs': [],
    'interaction_type': [],
    'mechanism': [],
    'severity': [],
    'clinical_effect': [],
    'evidence_level': [],
    'source_reference': []
}

# Major Ayurvedic herbs with documented interactions
interactions_data = [
    # ASHWAGANDHA (Withania somnifera)
    {
        'herb_name': 'Ashwagandha',
        'scientific_name': 'Withania somnifera',
        'interacting_drug_class': 'Immunosuppressants',
        'specific_drugs': 'Cyclosporine, Tacrolimus, Azathioprine',
        'interaction_type': 'Pharmacodynamic',
        'mechanism': 'Immune stimulation may counteract immunosuppression',
        'severity': 'Moderate',
        'clinical_effect': 'Reduced immunosuppressive effect',
        'evidence_level': 'Theoretical',
        'source_reference': 'MSK 2024'
    },
    {
        'herb_name': 'Ashwagandha',
        'scientific_name': 'Withania somnifera',
        'interacting_drug_class': 'Sedatives/CNS Depressants',
        'specific_drugs': 'Benzodiazepines, Barbiturates, Zolpidem',
        'interaction_type': 'Pharmacodynamic',
        'mechanism': 'Additive sedative effects',
        'severity': 'Moderate',
        'clinical_effect': 'Increased sedation, drowsiness, risk of falls',
        'evidence_level': 'Clinical',
        'source_reference': 'NCCIH 2023'
    },
    {
        'herb_name': 'Ashwagandha',
        'scientific_name': 'Withania somnifera',
        'interacting_drug_class': 'Thyroid Hormones',
        'specific_drugs': 'Levothyroxine',
        'interaction_type': 'Pharmacodynamic',
        'mechanism': 'May increase thyroid hormone levels',
        'severity': 'Moderate',
        'clinical_effect': 'Risk of hyperthyroidism',
        'evidence_level': 'Clinical',
        'source_reference': 'PubMed 2022'
    },
    {
        'herb_name': 'Ashwagandha',
        'scientific_name': 'Withania somnifera',
        'interacting_drug_class': 'CYP3A4 Substrates',
        'specific_drugs': 'Midazolam, Alprazolam, Simvastatin, Calcium channel blockers',
        'interaction_type': 'Pharmacokinetic',
        'mechanism': 'CYP3A4 induction - increases drug metabolism',
        'severity': 'Minor-Moderate',
        'clinical_effect': 'Reduced drug efficacy',
        'evidence_level': 'In vitro/Clinical',
        'source_reference': 'Frontiers Pharmacol 2022'
    },
    {
        'herb_name': 'Ashwagandha',
        'scientific_name': 'Withania somnifera',
        'interacting_drug_class': 'Antidiabetic Agents',
        'specific_drugs': 'Metformin, Insulin, Sulfonylureas',
        'interaction_type': 'Pharmacodynamic',
        'mechanism': 'Additive hypoglycemic effect',
        'severity': 'Moderate',
        'clinical_effect': 'Increased risk of hypoglycemia',
        'evidence_level': 'Clinical',
        'source_reference': 'Clinical studies 2021'
    },
    {
        'herb_name': 'Ashwagandha',
        'scientific_name': 'Withania somnifera',
        'interacting_drug_class': 'Antihypertensives',
        'specific_drugs': 'ACE inhibitors, ARBs, Beta-blockers',
        'interaction_type': 'Pharmacodynamic',
        'mechanism': 'Additive blood pressure lowering',
        'severity': 'Moderate',
        'clinical_effect': 'Risk of hypotension',
        'evidence_level': 'Theoretical/Clinical',
        'source_reference': 'Consensus 2024'
    },
    
    # TURMERIC (Curcuma longa)
    {
        'herb_name': 'Turmeric/Curcumin',
        'scientific_name': 'Curcuma longa',
        'interacting_drug_class': 'Anticoagulants/Antiplatelets',
        'specific_drugs': 'Warfarin, Aspirin, Clopidogrel, Heparin',
        'interaction_type': 'Pharmacodynamic',
        'mechanism': 'Antiplatelet effects, bleeding risk',
        'severity': 'Major',
        'clinical_effect': 'Increased bleeding risk',
        'evidence_level': 'Clinical',
        'source_reference': 'NCCIH 2023, PMC systematic reviews'
    },
    {
        'herb_name': 'Turmeric/Curcumin',
        'scientific_name': 'Curcuma longa',
        'interacting_drug_class': 'Hepatotoxic Drugs',
        'specific_drugs': 'Acetaminophen, Methotrexate, Statins at high dose',
        'interaction_type': 'Pharmacodynamic',
        'mechanism': 'Additive liver toxicity',
        'severity': 'Major',
        'clinical_effect': 'Increased hepatotoxicity risk',
        'evidence_level': 'Clinical',
        'source_reference': 'ScienceDirect 2024'
    },
    {
        'herb_name': 'Turmeric/Curcumin',
        'scientific_name': 'Curcuma longa',
        'interacting_drug_class': 'Antidiabetic Agents',
        'specific_drugs': 'Metformin, Insulin, Glyburide',
        'interaction_type': 'Pharmacodynamic',
        'mechanism': 'Additive hypoglycemic effect',
        'severity': 'Moderate',
        'clinical_effect': 'Increased hypoglycemia risk',
        'evidence_level': 'Clinical',
        'source_reference': 'Clinical trials 2023'
    },
    {
        'herb_name': 'Turmeric/Curcumin',
        'scientific_name': 'Curcuma longa',
        'interacting_drug_class': 'Chemotherapy Agents',
        'specific_drugs': 'Cyclophosphamide, Camptothecin',
        'interaction_type': 'Pharmacodynamic',
        'mechanism': 'Antioxidant effects may interfere with oxidative damage mechanism',
        'severity': 'Moderate',
        'clinical_effect': 'Potential reduced chemotherapy efficacy',
        'evidence_level': 'Preclinical/Theoretical',
        'source_reference': 'ASCO Post 2024'
    },
    {
        'herb_name': 'Turmeric/Curcumin',
        'scientific_name': 'Curcuma longa',
        'interacting_drug_class': 'CYP450 Substrates',
        'specific_drugs': 'Multiple CYP2C9, CYP3A4 substrates',
        'interaction_type': 'Pharmacokinetic',
        'mechanism': 'CYP enzyme inhibition',
        'severity': 'Minor-Moderate',
        'clinical_effect': 'Altered drug metabolism',
        'evidence_level': 'In vitro',
        'source_reference': 'PMC 2015'
    },
    
    # TRIPHALA
    {
        'herb_name': 'Triphala',
        'scientific_name': 'Terminalia chebula, Terminalia bellirica, Emblica officinalis',
        'interacting_drug_class': 'Antidiabetic Agents',
        'specific_drugs': 'Metformin, Insulin, Sulfonylureas',
        'interaction_type': 'Pharmacodynamic',
        'mechanism': 'May lower blood glucose',
        'severity': 'Moderate',
        'clinical_effect': 'Risk of hypoglycemia',
        'evidence_level': 'Clinical',
        'source_reference': 'Ayurvedic pharmacology 2023'
    },
    {
        'herb_name': 'Triphala',
        'scientific_name': 'Terminalia chebula, Terminalia bellirica, Emblica officinalis',
        'interacting_drug_class': 'Anticoagulants',
        'specific_drugs': 'Warfarin',
        'interaction_type': 'Pharmacodynamic',
        'mechanism': 'High vitamin C content may affect INR',
        'severity': 'Moderate',
        'clinical_effect': 'Altered anticoagulation',
        'evidence_level': 'Theoretical',
        'source_reference': 'Clinical consideration 2022'
    },
    {
        'herb_name': 'Triphala',
        'scientific_name': 'Terminalia chebula, Terminalia bellirica, Emblica officinalis',
        'interacting_drug_class': 'Laxatives',
        'specific_drugs': 'Senna, Bisacodyl, Docusate',
        'interaction_type': 'Pharmacodynamic',
        'mechanism': 'Additive laxative effect',
        'severity': 'Moderate',
        'clinical_effect': 'Excessive diarrhea, electrolyte imbalance',
        'evidence_level': 'Clinical',
        'source_reference': 'Clinical reports'
    },
    
    # BRAHMI (Bacopa monnieri)
    {
        'herb_name': 'Brahmi',
        'scientific_name': 'Bacopa monnieri',
        'interacting_drug_class': 'Anticholinergics',
        'specific_drugs': 'Oxybutynin, Tolterodine, Diphenhydramine',
        'interaction_type': 'Pharmacodynamic',
        'mechanism': 'Cholinergic activity opposes anticholinergic effects',
        'severity': 'Moderate',
        'clinical_effect': 'Reduced drug efficacy',
        'evidence_level': 'Theoretical',
        'source_reference': 'Pharmacology reviews 2023'
    },
    {
        'herb_name': 'Brahmi',
        'scientific_name': 'Bacopa monnieri',
        'interacting_drug_class': 'Thyroid Hormones',
        'specific_drugs': 'Levothyroxine',
        'interaction_type': 'Pharmacodynamic',
        'mechanism': 'May increase T4 levels',
        'severity': 'Minor-Moderate',
        'clinical_effect': 'Risk of hyperthyroidism',
        'evidence_level': 'Animal studies',
        'source_reference': 'Preclinical 2022'
    },
    {
        'herb_name': 'Brahmi',
        'scientific_name': 'Bacopa monnieri',
        'interacting_drug_class': 'CYP450 Substrates',
        'specific_drugs': 'CYP2C9, CYP2C19, CYP3A4 substrates',
        'interaction_type': 'Pharmacokinetic',
        'mechanism': 'CYP enzyme inhibition',
        'severity': 'Minor',
        'clinical_effect': 'Altered drug levels',
        'evidence_level': 'In vitro',
        'source_reference': 'In vitro studies 2021'
    },
    
    # HOLY BASIL (Tulsi - Ocimum sanctum)
    {
        'herb_name': 'Holy Basil (Tulsi)',
        'scientific_name': 'Ocimum sanctum',
        'interacting_drug_class': 'Antidiabetic Agents',
        'specific_drugs': 'Insulin, Metformin, Sulfonylureas',
        'interaction_type': 'Pharmacodynamic',
        'mechanism': 'Hypoglycemic effect',
        'severity': 'Moderate',
        'clinical_effect': 'Risk of hypoglycemia',
        'evidence_level': 'Clinical',
        'source_reference': 'Clinical studies 2023'
    },
    {
        'herb_name': 'Holy Basil (Tulsi)',
        'scientific_name': 'Ocimum sanctum',
        'interacting_drug_class': 'Anticoagulants',
        'specific_drugs': 'Warfarin, Aspirin',
        'interaction_type': 'Pharmacodynamic',
        'mechanism': 'Antiplatelet activity',
        'severity': 'Moderate',
        'clinical_effect': 'Increased bleeding risk',
        'evidence_level': 'Theoretical/Preclinical',
        'source_reference': 'Pharmacology 2022'
    },
    
    # GUGGUL (Commiphora mukul)
    {
        'herb_name': 'Guggul',
        'scientific_name': 'Commiphora mukul',
        'interacting_drug_class': 'Thyroid Hormones',
        'specific_drugs': 'Levothyroxine',
        'interaction_type': 'Pharmacodynamic',
        'mechanism': 'May increase thyroid activity',
        'severity': 'Moderate',
        'clinical_effect': 'Risk of hyperthyroidism',
        'evidence_level': 'Clinical',
        'source_reference': 'Clinical reports 2022'
    },
    {
        'herb_name': 'Guggul',
        'scientific_name': 'Commiphora mukul',
        'interacting_drug_class': 'Anticoagulants',
        'specific_drugs': 'Warfarin',
        'interaction_type': 'Pharmacodynamic',
        'mechanism': 'May potentiate anticoagulant effects',
        'severity': 'Major',
        'clinical_effect': 'Increased bleeding risk',
        'evidence_level': 'Case reports',
        'source_reference': 'Clinical case reports'
    },
    {
        'herb_name': 'Guggul',
        'scientific_name': 'Commiphora mukul',
        'interacting_drug_class': 'CYP3A4 Substrates',
        'specific_drugs': 'Diltiazem, Propranolol, Midazolam',
        'interaction_type': 'Pharmacokinetic',
        'mechanism': 'CYP3A4 induction',
        'severity': 'Moderate',
        'clinical_effect': 'Reduced drug efficacy',
        'evidence_level': 'In vitro/Clinical',
        'source_reference': 'Pharmacokinetic studies 2021'
    },
    
    # SHANKHPUSHPI (Convolvulus pluricaulis)
    {
        'herb_name': 'Shankhpushpi',
        'scientific_name': 'Convolvulus pluricaulis',
        'interacting_drug_class': 'Antiepileptics',
        'specific_drugs': 'Phenytoin',
        'interaction_type': 'Pharmacokinetic',
        'mechanism': 'Decreased phenytoin levels (documented case)',
        'severity': 'Major',
        'clinical_effect': 'Loss of seizure control',
        'evidence_level': 'Case report',
        'source_reference': 'Epilepsia case report 2002'
    },
    {
        'herb_name': 'Shankhpushpi',
        'scientific_name': 'Convolvulus pluricaulis',
        'interacting_drug_class': 'Sedatives',
        'specific_drugs': 'Benzodiazepines, Z-drugs',
        'interaction_type': 'Pharmacodynamic',
        'mechanism': 'Additive sedative effects',
        'severity': 'Moderate',
        'clinical_effect': 'Increased sedation',
        'evidence_level': 'Theoretical',
        'source_reference': 'Pharmacology 2023'
    },
    
    # GINGER (Zingiber officinale)
    {
        'herb_name': 'Ginger',
        'scientific_name': 'Zingiber officinale',
        'interacting_drug_class': 'Anticoagulants/Antiplatelets',
        'specific_drugs': 'Warfarin, Aspirin, Clopidogrel',
        'interaction_type': 'Pharmacodynamic',
        'mechanism': 'Antiplatelet effects',
        'severity': 'Moderate',
        'clinical_effect': 'Increased bleeding risk',
        'evidence_level': 'Clinical',
        'source_reference': 'NCCIH 2023, PMC reviews'
    },
    {
        'herb_name': 'Ginger',
        'scientific_name': 'Zingiber officinale',
        'interacting_drug_class': 'Antidiabetic Agents',
        'specific_drugs': 'Insulin, Metformin',
        'interaction_type': 'Pharmacodynamic',
        'mechanism': 'May lower blood glucose',
        'severity': 'Moderate',
        'clinical_effect': 'Risk of hypoglycemia',
        'evidence_level': 'Clinical',
        'source_reference': 'Clinical trials 2022'
    },
    
    # BOSWELLIA (Shallaki - Boswellia serrata)
    {
        'herb_name': 'Boswellia (Shallaki)',
        'scientific_name': 'Boswellia serrata',
        'interacting_drug_class': 'Immunosuppressants',
        'specific_drugs': 'Cyclosporine, Tacrolimus',
        'interaction_type': 'Pharmacodynamic',
        'mechanism': 'Immune modulation may interfere',
        'severity': 'Moderate',
        'clinical_effect': 'Altered immunosuppression',
        'evidence_level': 'Theoretical',
        'source_reference': 'MSK database'
    },
    {
        'herb_name': 'Boswellia (Shallaki)',
        'scientific_name': 'Boswellia serrata',
        'interacting_drug_class': 'NSAIDs',
        'specific_drugs': 'Ibuprofen, Naproxen, Diclofenac',
        'interaction_type': 'Pharmacodynamic',
        'mechanism': 'Additive anti-inflammatory effects, GI risk',
        'severity': 'Minor-Moderate',
        'clinical_effect': 'Increased GI side effects',
        'evidence_level': 'Theoretical',
        'source_reference': 'Clinical pharmacology 2023'
    },
    
    # NEEM (Azadirachta indica)
    {
        'herb_name': 'Neem',
        'scientific_name': 'Azadirachta indica',
        'interacting_drug_class': 'Immunosuppressants',
        'specific_drugs': 'Cyclosporine, Prednisone, Azathioprine',
        'interaction_type': 'Pharmacodynamic',
        'mechanism': 'Immune stimulation may counteract',
        'severity': 'Moderate',
        'clinical_effect': 'Reduced immunosuppressive effect',
        'evidence_level': 'Theoretical',
        'source_reference': 'Herbal medicine reviews 2022'
    },
    {
        'herb_name': 'Neem',
        'scientific_name': 'Azadirachta indica',
        'interacting_drug_class': 'Antidiabetic Agents',
        'specific_drugs': 'Insulin, Metformin, Sulfonylureas',
        'interaction_type': 'Pharmacodynamic',
        'mechanism': 'Hypoglycemic effect',
        'severity': 'Moderate',
        'clinical_effect': 'Risk of hypoglycemia',
        'evidence_level': 'Clinical',
        'source_reference': 'Clinical studies 2023'
    },
    {
        'herb_name': 'Neem',
        'scientific_name': 'Azadirachta indica',
        'interacting_drug_class': 'Lithium',
        'specific_drugs': 'Lithium carbonate',
        'interaction_type': 'Pharmacokinetic',
        'mechanism': 'Diuretic effect may increase lithium levels',
        'severity': 'Major',
        'clinical_effect': 'Risk of lithium toxicity',
        'evidence_level': 'Theoretical',
        'source_reference': 'Drug interaction databases'
    },
    
    # ARJUNA (Terminalia arjuna)
    {
        'herb_name': 'Arjuna',
        'scientific_name': 'Terminalia arjuna',
        'interacting_drug_class': 'Antihypertensives',
        'specific_drugs': 'ACE inhibitors, Beta-blockers, Calcium channel blockers',
        'interaction_type': 'Pharmacodynamic',
        'mechanism': 'Additive blood pressure lowering',
        'severity': 'Moderate',
        'clinical_effect': 'Risk of hypotension',
        'evidence_level': 'Clinical',
        'source_reference': 'Clinical trials 2022'
    },
    {
        'herb_name': 'Arjuna',
        'scientific_name': 'Terminalia arjuna',
        'interacting_drug_class': 'Anticoagulants',
        'specific_drugs': 'Warfarin, Aspirin',
        'interaction_type': 'Pharmacodynamic',
        'mechanism': 'Antiplatelet activity',
        'severity': 'Moderate',
        'clinical_effect': 'Increased bleeding risk',
        'evidence_level': 'Theoretical',
        'source_reference': 'Pharmacology reviews 2023'
    },
    {
        'herb_name': 'Arjuna',
        'scientific_name': 'Terminalia arjuna',
        'interacting_drug_class': 'Cardiac Glycosides',
        'specific_drugs': 'Digoxin',
        'interaction_type': 'Pharmacodynamic',
        'mechanism': 'May enhance cardiac effects',
        'severity': 'Moderate',
        'clinical_effect': 'Risk of digoxin toxicity',
        'evidence_level': 'Theoretical',
        'source_reference': 'Cardiology reviews 2022'
    },
    
    # SHATAVARI (Asparagus racemosus)
    {
        'herb_name': 'Shatavari',
        'scientific_name': 'Asparagus racemosus',
        'interacting_drug_class': 'Diuretics',
        'specific_drugs': 'Furosemide, Hydrochlorothiazide',
        'interaction_type': 'Pharmacodynamic',
        'mechanism': 'Additive diuretic effect',
        'severity': 'Moderate',
        'clinical_effect': 'Dehydration, electrolyte imbalance',
        'evidence_level': 'Theoretical',
        'source_reference': 'Pharmacology 2023'
    },
    {
        'herb_name': 'Shatavari',
        'scientific_name': 'Asparagus racemosus',
        'interacting_drug_class': 'Hormone Therapy',
        'specific_drugs': 'Estrogen replacement, Tamoxifen',
        'interaction_type': 'Pharmacodynamic',
        'mechanism': 'Phytoestrogen activity',
        'severity': 'Moderate',
        'clinical_effect': 'May interfere with hormone therapy',
        'evidence_level': 'Theoretical',
        'source_reference': 'Oncology reviews 2022'
    },
    
    # AYUSH-64 (Polyherbal formulation)
    {
        'herb_name': 'AYUSH-64',
        'scientific_name': 'Alstonia scholaris, Picrorhiza kurroa, Swertia chirata, Caesalpinia crista',
        'interacting_drug_class': 'CYP2C8 Substrates',
        'specific_drugs': 'Repaglinide, Paclitaxel, Rosiglitazone',
        'interaction_type': 'Pharmacokinetic',
        'mechanism': 'CYP2C8 inhibition',
        'severity': 'Moderate',
        'clinical_effect': 'Increased drug levels, toxicity risk',
        'evidence_level': 'In vitro',
        'source_reference': 'Frontiers Pharmacol 2022'
    },
    {
        'herb_name': 'AYUSH-64',
        'scientific_name': 'Alstonia scholaris, Picrorhiza kurroa, Swertia chirata, Caesalpinia crista',
        'interacting_drug_class': 'Antiviral Agents',
        'specific_drugs': 'Remdesivir',
        'interaction_type': 'Pharmacokinetic',
        'mechanism': 'Potential CYP enzyme interaction',
        'severity': 'Minor-Moderate',
        'clinical_effect': 'Altered antiviral drug levels',
        'evidence_level': 'In vitro',
        'source_reference': 'COVID-19 studies 2022'
    },
]

# Populate the dictionary
for interaction in interactions_data:
    for key in ayurvedic_known_interactions.keys():
        ayurvedic_known_interactions[key].append(interaction[key])

# Create DataFrame and save
df_known = pd.DataFrame(ayurvedic_known_interactions)
df_known.to_csv('ayurvedic_known_interactions.csv', index=False)
print(f"✓ Created ayurvedic_known_interactions.csv with {len(df_known)} interactions")

# ============================================================================
# AYURVEDIC PHARMACOLOGICAL PROFILES (Unknown Risk Simulator Mapping)
# Based on traditional Ayurvedic pharmacology (Dravyaguna) and clinical use
# ============================================================================

ayurvedic_profiles = {
    'herbs': []
}

# Comprehensive herb profiles with pharmacological-like classifications
herb_profiles = [
    {
        'herb_name': 'Ashwagandha',
        'scientific_name': 'Withania somnifera',
        'common_indications': ['stress', 'anxiety', 'immunity', 'energy', 'sleep'],
        'ayurvedic_rasa': ['bitter', 'astringent', 'sweet'],
        'ayurvedic_virya': 'heating',
        'ayurvedic_category': 'Rasayana (rejuvenator), Balya (strength)',
        'pharmacological_profile': {
            'sedative_like': 0.7,
            'anxiolytic_like': 0.8,
            'immunomodulator': 0.7,
            'adaptogenic': 0.9,
            'thyroid_stimulant': 0.5,
            'hypotensive': 0.4,
            'hypoglycemic': 0.5
        },
        'safety_concerns': ['sedation', 'thyroid effects', 'immunomodulation'],
        'contraindicated_conditions': ['hyperthyroidism', 'severe pregnancy complications'],
        'organ_tropism': ['nervous_system', 'immune_system', 'endocrine']
    },
    {
        'herb_name': 'Turmeric',
        'scientific_name': 'Curcuma longa',
        'common_indications': ['inflammation', 'pain', 'immunity', 'digestion'],
        'ayurvedic_rasa': ['bitter', 'pungent'],
        'ayurvedic_virya': 'heating',
        'ayurvedic_category': 'Varnya (complexion), Kushthagna (skin diseases)',
        'pharmacological_profile': {
            'anti_inflammatory': 0.9,
            'antiplatelet': 0.6,
            'hepatoprotective': 0.7,
            'hepatotoxic_high_dose': 0.4,
            'hypoglycemic': 0.5,
            'immunomodulator': 0.6
        },
        'safety_concerns': ['bleeding risk', 'hepatotoxicity at high doses', 'gallbladder stimulation'],
        'contraindicated_conditions': ['bile duct obstruction', 'active bleeding'],
        'organ_tropism': ['liver', 'blood', 'digestive_system']
    },
    {
        'herb_name': 'Brahmi',
        'scientific_name': 'Bacopa monnieri',
        'common_indications': ['memory', 'cognition', 'anxiety', 'mental clarity'],
        'ayurvedic_rasa': ['bitter'],
        'ayurvedic_virya': 'cooling',
        'ayurvedic_category': 'Medhya (brain tonic)',
        'pharmacological_profile': {
            'cognitive_enhancer': 0.8,
            'anxiolytic_like': 0.6,
            'cholinergic': 0.5,
            'thyroid_stimulant': 0.4,
            'sedative_like': 0.3
        },
        'safety_concerns': ['thyroid effects', 'GI upset', 'bradycardia'],
        'contraindicated_conditions': ['bradycardia', 'hypothyroidism'],
        'organ_tropism': ['nervous_system', 'endocrine']
    },
    {
        'herb_name': 'Triphala',
        'scientific_name': 'Terminalia chebula + Terminalia bellirica + Emblica officinalis',
        'common_indications': ['digestion', 'constipation', 'detox', 'immunity'],
        'ayurvedic_rasa': ['all six tastes'],
        'ayurvedic_virya': 'neutral',
        'ayurvedic_category': 'Rasayana, Anulomana (laxative)',
        'pharmacological_profile': {
            'laxative': 0.7,
            'antioxidant': 0.8,
            'hypoglycemic': 0.5,
            'anticoagulant_mild': 0.3,
            'immunomodulator': 0.6
        },
        'safety_concerns': ['diarrhea', 'electrolyte imbalance', 'INR alterations'],
        'contraindicated_conditions': ['diarrhea', 'intestinal obstruction'],
        'organ_tropism': ['digestive_system', 'colon']
    },
    {
        'herb_name': 'Holy Basil (Tulsi)',
        'scientific_name': 'Ocimum sanctum',
        'common_indications': ['immunity', 'cough', 'stress', 'fever'],
        'ayurvedic_rasa': ['pungent', 'bitter'],
        'ayurvedic_virya': 'heating',
        'ayurvedic_category': 'Kasa-swasahara (respiratory), Rasayana',
        'pharmacological_profile': {
            'adaptogenic': 0.7,
            'immunomodulator': 0.8,
            'hypoglycemic': 0.6,
            'antiplatelet': 0.4,
            'antitussive': 0.5
        },
        'safety_concerns': ['hypoglycemia', 'bleeding risk', 'fertility effects'],
        'contraindicated_conditions': ['pregnancy', 'surgery within 2 weeks'],
        'organ_tropism': ['respiratory_system', 'immune_system']
    },
    {
        'herb_name': 'Guggul',
        'scientific_name': 'Commiphora mukul',
        'common_indications': ['cholesterol', 'arthritis', 'weight loss'],
        'ayurvedic_rasa': ['bitter', 'pungent', 'astringent'],
        'ayurvedic_virya': 'heating',
        'ayurvedic_category': 'Medohara (fat reducer), Amapachana',
        'pharmacological_profile': {
            'hypolipidemic': 0.7,
            'anti_inflammatory': 0.6,
            'thyroid_stimulant': 0.6,
            'anticoagulant': 0.5,
            'hepatotoxic_concern': 0.3
        },
        'safety_concerns': ['bleeding risk', 'thyroid effects', 'hepatotoxicity'],
        'contraindicated_conditions': ['hyperthyroidism', 'bleeding disorders', 'liver disease'],
        'organ_tropism': ['liver', 'thyroid', 'cardiovascular']
    },
    {
        'herb_name': 'Shankhpushpi',
        'scientific_name': 'Convolvulus pluricaulis',
        'common_indications': ['memory', 'anxiety', 'sleep', 'mental health'],
        'ayurvedic_rasa': ['sweet', 'bitter'],
        'ayurvedic_virya': 'cooling',
        'ayurvedic_category': 'Medhya, Nidrajanana (sleep inducer)',
        'pharmacological_profile': {
            'sedative_like': 0.6,
            'anxiolytic_like': 0.7,
            'cognitive_enhancer': 0.6,
            'antiepileptic_interaction': 0.7
        },
        'safety_concerns': ['sedation', 'antiepileptic drug interactions'],
        'contraindicated_conditions': ['epilepsy on phenytoin'],
        'organ_tropism': ['nervous_system']
    },
    {
        'herb_name': 'Ginger',
        'scientific_name': 'Zingiber officinale',
        'common_indications': ['nausea', 'digestion', 'inflammation', 'pain'],
        'ayurvedic_rasa': ['pungent'],
        'ayurvedic_virya': 'heating',
        'ayurvedic_category': 'Deepana (appetizer), Shothahara (anti-inflammatory)',
        'pharmacological_profile': {
            'antiemetic': 0.8,
            'antiplatelet': 0.5,
            'anti_inflammatory': 0.6,
            'hypoglycemic': 0.4,
            'prokinetic': 0.6
        },
        'safety_concerns': ['bleeding risk', 'heartburn', 'hypoglycemia'],
        'contraindicated_conditions': ['active bleeding', 'gallstones'],
        'organ_tropism': ['digestive_system', 'blood']
    },
    {
        'herb_name': 'Boswellia (Shallaki)',
        'scientific_name': 'Boswellia serrata',
        'common_indications': ['arthritis', 'inflammation', 'pain'],
        'ayurvedic_rasa': ['bitter', 'pungent', 'astringent'],
        'ayurvedic_virya': 'cooling',
        'ayurvedic_category': 'Sandhigata vataghna (joint disorders)',
        'pharmacological_profile': {
            'anti_inflammatory': 0.9,
            'analgesic': 0.6,
            'immunomodulator': 0.5,
            'GI_irritant': 0.4
        },
        'safety_concerns': ['GI upset', 'nausea', 'immunomodulation'],
        'contraindicated_conditions': ['pregnancy', 'autoimmune on immunosuppressants'],
        'organ_tropism': ['joints', 'digestive_system']
    },
    {
        'herb_name': 'Neem',
        'scientific_name': 'Azadirachta indica',
        'common_indications': ['skin conditions', 'immunity', 'blood sugar', 'parasites'],
        'ayurvedic_rasa': ['bitter'],
        'ayurvedic_virya': 'cooling',
        'ayurvedic_category': 'Kushthagna, Krimighna (antiparasitic)',
        'pharmacological_profile': {
            'antimicrobial': 0.7,
            'immunomodulator': 0.7,
            'hypoglycemic': 0.7,
            'contraceptive': 0.5,
            'hepatotoxic_concern': 0.3
        },
        'safety_concerns': ['hypoglycemia', 'fertility effects', 'hepatotoxicity', 'immunomodulation'],
        'contraindicated_conditions': ['pregnancy', 'infertility treatment', 'organ transplant'],
        'organ_tropism': ['skin', 'liver', 'immune_system']
    },
    {
        'herb_name': 'Arjuna',
        'scientific_name': 'Terminalia arjuna',
        'common_indications': ['heart health', 'hypertension', 'cholesterol'],
        'ayurvedic_rasa': ['astringent'],
        'ayurvedic_virya': 'cooling',
        'ayurvedic_category': 'Hridya (cardiotonic)',
        'pharmacological_profile': {
            'cardiotonic': 0.8,
            'hypotensive': 0.7,
            'hypolipidemic': 0.6,
            'antiplatelet': 0.4,
            'inotropic': 0.5
        },
        'safety_concerns': ['hypotension', 'bleeding risk', 'cardiac glycoside interaction'],
        'contraindicated_conditions': ['severe hypotension', 'bradycardia'],
        'organ_tropism': ['cardiovascular']
    },
    {
        'herb_name': 'Shatavari',
        'scientific_name': 'Asparagus racemosus',
        'common_indications': ['female reproductive health', 'lactation', 'immunity', 'ulcers'],
        'ayurvedic_rasa': ['sweet', 'bitter'],
        'ayurvedic_virya': 'cooling',
        'ayurvedic_category': 'Rasayana, Stanyajanana (galactagogue)',
        'pharmacological_profile': {
            'galactagogue': 0.7,
            'phytoestrogenic': 0.5,
            'immunomodulator': 0.6,
            'diuretic': 0.5,
            'gastroprotective': 0.6
        },
        'safety_concerns': ['hormone-sensitive conditions', 'diuretic effect', 'allergy'],
        'contraindicated_conditions': ['estrogen-dependent tumors', 'kidney stones'],
        'organ_tropism': ['reproductive_system', 'kidney', 'digestive_system']
    },
    {
        'herb_name': 'Guduchi',
        'scientific_name': 'Tinospora cordifolia',
        'common_indications': ['immunity', 'fever', 'diabetes', 'liver health'],
        'ayurvedic_rasa': ['bitter', 'astringent'],
        'ayurvedic_virya': 'heating',
        'ayurvedic_category': 'Rasayana, Jvaraghna (antipyretic)',
        'pharmacological_profile': {
            'immunomodulator': 0.9,
            'hepatoprotective': 0.7,
            'hypoglycemic': 0.6,
            'antipyretic': 0.6,
            'adaptogenic': 0.5
        },
        'safety_concerns': ['hypoglycemia', 'immunomodulation', 'autoimmune activation'],
        'contraindicated_conditions': ['autoimmune diseases', 'organ transplant'],
        'organ_tropism': ['immune_system', 'liver']
    },
    {
        'herb_name': 'Haritaki',
        'scientific_name': 'Terminalia chebula',
        'common_indications': ['digestion', 'constipation', 'detox', 'rejuvenation'],
        'ayurvedic_rasa': ['all except salty'],
        'ayurvedic_virya': 'heating',
        'ayurvedic_category': 'Rasayana, Anulomana',
        'pharmacological_profile': {
            'laxative': 0.8,
            'antioxidant': 0.7,
            'hypoglycemic': 0.5,
            'hypolipidemic': 0.5,
            'antimicrobial': 0.6
        },
        'safety_concerns': ['diarrhea', 'dehydration', 'hypoglycemia'],
        'contraindicated_conditions': ['pregnancy', 'diarrhea', 'dehydration'],
        'organ_tropism': ['digestive_system', 'colon']
    },
    {
        'herb_name': 'Amla',
        'scientific_name': 'Emblica officinalis',
        'common_indications': ['immunity', 'vitamin C', 'hair health', 'digestion'],
        'ayurvedic_rasa': ['sour', 'sweet', 'bitter', 'pungent', 'astringent'],
        'ayurvedic_virya': 'cooling',
        'ayurvedic_category': 'Rasayana, Varnya',
        'pharmacological_profile': {
            'antioxidant': 0.9,
            'immunomodulator': 0.7,
            'hypoglycemic': 0.5,
            'hypolipidemic': 0.6,
            'anticoagulant_mild': 0.3
        },
        'safety_concerns': ['bleeding risk (high vitamin C)', 'INR changes', 'diarrhea'],
        'contraindicated_conditions': ['bleeding disorders'],
        'organ_tropism': ['immune_system', 'digestive_system']
    },
    {
        'herb_name': 'Punarnava',
        'scientific_name': 'Boerhavia diffusa',
        'common_indications': ['kidney health', 'edema', 'liver health', 'diuretic'],
        'ayurvedic_rasa': ['sweet', 'bitter'],
        'ayurvedic_virya': 'cooling',
        'ayurvedic_category': 'Shodhana (diuretic), Yakriduttejaka (hepatoprotective)',
        'pharmacological_profile': {
            'diuretic': 0.8,
            'hepatoprotective': 0.6,
            'anti_inflammatory': 0.5,
            'immunomodulator': 0.5
        },
        'safety_concerns': ['electrolyte imbalance', 'dehydration', 'lithium interaction'],
        'contraindicated_conditions': ['severe dehydration', 'lithium therapy'],
        'organ_tropism': ['kidney', 'liver']
    },
    {
        'herb_name': 'Licorice (Yashtimadhu)',
        'scientific_name': 'Glycyrrhiza glabra',
        'common_indications': ['cough', 'ulcers', 'adrenal support', 'throat'],
        'ayurvedic_rasa': ['sweet'],
        'ayurvedic_virya': 'cooling',
        'ayurvedic_category': 'Kasa-swasahara, Vayasthapana (anti-aging)',
        'pharmacological_profile': {
            'demulcent': 0.7,
            'anti_inflammatory': 0.6,
            'cortisol_like': 0.7,
            'hypertensive': 0.8,
            'hypokalemic': 0.7,
            'mineralocorticoid': 0.6
        },
        'safety_concerns': ['hypertension', 'hypokalemia', 'edema', 'cardiac arrhythmia'],
        'contraindicated_conditions': ['hypertension', 'heart failure', 'hypokalemia', 'pregnancy'],
        'organ_tropism': ['respiratory_system', 'adrenal', 'cardiovascular']
    },
    {
        'herb_name': 'Fenugreek (Methi)',
        'scientific_name': 'Trigonella foenum-graecum',
        'common_indications': ['diabetes', 'lactation', 'digestion', 'cholesterol'],
        'ayurvedic_rasa': ['bitter', 'pungent'],
        'ayurvedic_virya': 'heating',
        'ayurvedic_category': 'Stanyajanana, Medhohara',
        'pharmacological_profile': {
            'hypoglycemic': 0.8,
            'hypolipidemic': 0.6,
            'galactagogue': 0.7,
            'anticoagulant': 0.4,
            'uterotonic': 0.5
        },
        'safety_concerns': ['hypoglycemia', 'bleeding risk', 'uterine stimulation'],
        'contraindicated_conditions': ['pregnancy', 'bleeding disorders'],
        'organ_tropism': ['pancreas', 'blood', 'reproductive_system']
    },
    {
        'herb_name': 'Moringa',
        'scientific_name': 'Moringa oleifera',
        'common_indications': ['nutrition', 'immunity', 'lactation', 'energy'],
        'ayurvedic_rasa': ['pungent'],
        'ayurvedic_virya': 'heating',
        'ayurvedic_category': 'Balya, Stanyajanana',
        'pharmacological_profile': {
            'nutritive': 0.8,
            'hypoglycemic': 0.5,
            'hypotensive': 0.5,
            'hypolipidemic': 0.5,
            'galactagogue': 0.6
        },
        'safety_concerns': ['hypoglycemia', 'hypotension', 'laxative effect'],
        'contraindicated_conditions': ['hypothyroidism (potential)'],
        'organ_tropism': ['general_nutrition', 'immune_system']
    },
    {
        'herb_name': 'Bala',
        'scientific_name': 'Sida cordifolia',
        'common_indications': ['strength', 'immunity', 'nervous system', 'energy'],
        'ayurvedic_rasa': ['sweet'],
        'ayurvedic_virya': 'cooling',
        'ayurvedic_category': 'Balya, Rasayana',
        'pharmacological_profile': {
            'stimulant': 0.6,
            'cardiotonic': 0.5,
            'adaptogenic': 0.5,
            'ephedrine_like': 0.7
        },
        'safety_concerns': ['cardiovascular stimulation', 'hypertension', 'insomnia'],
        'contraindicated_conditions': ['hypertension', 'heart disease', 'anxiety'],
        'organ_tropism': ['cardiovascular', 'nervous_system']
    },
    {
        'herb_name': 'Vacha',
        'scientific_name': 'Acorus calamus',
        'common_indications': ['cognitive', 'speech', 'epilepsy', 'mental clarity'],
        'ayurvedic_rasa': ['pungent', 'bitter'],
        'ayurvedic_virya': 'heating',
        'ayurvedic_category': 'Medhya',
        'pharmacological_profile': {
            'cognitive_enhancer': 0.7,
            'sedative_like': 0.4,
            'carcinogenic_concern': 0.6,
            'hepatotoxic_concern': 0.5
        },
        'safety_concerns': ['carcinogenicity (beta-asarone)', 'hepatotoxicity', 'banned in some countries'],
        'contraindicated_conditions': ['pregnancy', 'liver disease'],
        'organ_tropism': ['nervous_system', 'liver']
    },
    {
        'herb_name': 'Jatamansi',
        'scientific_name': 'Nardostachys jatamansi',
        'common_indications': ['anxiety', 'sleep', 'mental health', 'hair growth'],
        'ayurvedic_rasa': ['bitter', 'astringent', 'sweet'],
        'ayurvedic_virya': 'cooling',
        'ayurvedic_category': 'Medhya, Nidrajanana',
        'pharmacological_profile': {
            'sedative_like': 0.7,
            'anxiolytic_like': 0.8,
            'hypotensive': 0.4,
            'hepatoprotective': 0.5
        },
        'safety_concerns': ['sedation', 'hypotension', 'CNS depression'],
        'contraindicated_conditions': ['severe hypotension', 'CNS depressant use'],
        'organ_tropism': ['nervous_system', 'cardiovascular']
    },
    {
        'herb_name': 'Vidari Kanda',
        'scientific_name': 'Pueraria tuberosa',
        'common_indications': ['strength', 'lactation', 'aphrodisiac', 'rejuvenation'],
        'ayurvedic_rasa': ['sweet'],
        'ayurvedic_virya': 'cooling',
        'ayurvedic_category': 'Balya, Vrishya (aphrodisiac), Stanyajanana',
        'pharmacological_profile': {
            'phytoestrogenic': 0.6,
            'galactagogue': 0.6,
            'nutritive': 0.7,
            'adaptogenic': 0.5
        },
        'safety_concerns': ['hormone-sensitive conditions', 'estrogenic effects'],
        'contraindicated_conditions': ['estrogen-dependent cancers'],
        'organ_tropism': ['reproductive_system', 'endocrine']
    },
    {
        'herb_name': 'Gokshura',
        'scientific_name': 'Tribulus terrestris',
        'common_indications': ['kidney stones', 'urinary health', 'libido', 'strength'],
        'ayurvedic_rasa': ['sweet'],
        'ayurvedic_virya': 'cooling',
        'ayurvedic_category': 'Vrishya, Mutrala (diuretic)',
        'pharmacological_profile': {
            'diuretic': 0.6,
            'aphrodisiac': 0.5,
            'testosterone_like': 0.4,
            'hypoglycemic': 0.4
        },
        'safety_concerns': ['hormonal effects', 'hypoglycemia', 'photosensitivity'],
        'contraindicated_conditions': ['hormone-sensitive cancers', 'prostate conditions'],
        'organ_tropism': ['kidney', 'reproductive_system']
    },
    {
        'herb_name': 'Kalmegh',
        'scientific_name': 'Andrographis paniculata',
        'common_indications': ['fever', 'immunity', 'liver health', 'infection'],
        'ayurvedic_rasa': ['bitter'],
        'ayurvedic_virya': 'cooling',
        'ayurvedic_category': 'Jvaraghna, Krimighna',
        'pharmacological_profile': {
            'immunomodulator': 0.8,
            'hepatoprotective': 0.7,
            'antipyretic': 0.7,
            'antiviral': 0.6,
            'antiplatelet': 0.4
        },
        'safety_concerns': ['bleeding risk', 'hypotension', 'fertility effects', 'immunomodulation'],
        'contraindicated_conditions': ['pregnancy', 'autoimmune diseases', 'bleeding disorders'],
        'organ_tropism': ['immune_system', 'liver']
    },
    {
        'herb_name': 'Manjistha',
        'scientific_name': 'Rubia cordifolia',
        'common_indications': ['skin conditions', 'blood purification', 'menstrual health'],
        'ayurvedic_rasa': ['bitter', 'astringent', 'sweet'],
        'ayurvedic_virya': 'cooling',
        'ayurvedic_category': 'Raktashodhaka (blood purifier), Varnya',
        'pharmacological_profile': {
            'blood_purifier': 0.6,
            'anti_inflammatory': 0.6,
            'antiplatelet': 0.4,
            'hepatoprotective': 0.5,
            'nephrotoxic_concern': 0.3
        },
        'safety_concerns': ['bleeding risk', 'kidney stones', 'urine discoloration'],
        'contraindicated_conditions': ['kidney disease', 'bleeding disorders'],
        'organ_tropism': ['blood', 'skin', 'kidney']
    },
    {
        'herb_name': 'Pippali',
        'scientific_name': 'Piper longum',
        'common_indications': ['respiratory', 'digestion', 'bioavailability enhancer'],
        'ayurvedic_rasa': ['pungent'],
        'ayurvedic_virya': 'heating',
        'ayurvedic_category': 'Kasa-swasahara, Deepana, Yogavahi (bioenhancer)',
        'pharmacological_profile': {
            'bioenhancer': 0.9,
            'bronchodilator': 0.6,
            'hepatoprotective': 0.5,
            'drug_absorption_enhancer': 0.8
        },
        'safety_concerns': ['increases drug absorption', 'GI irritation', 'drug interaction potential'],
        'contraindicated_conditions': ['peptic ulcer', 'acid reflux'],
        'organ_tropism': ['digestive_system', 'respiratory_system', 'bioavailability']
    },
    {
        'herb_name': 'Chyawanprash',
        'scientific_name': 'Polyherbal formulation (Amla-based)',
        'common_indications': ['immunity', 'energy', 'respiratory health', 'rejuvenation'],
        'ayurvedic_rasa': ['sweet', 'sour'],
        'ayurvedic_virya': 'heating',
        'ayurvedic_category': 'Rasayana',
        'pharmacological_profile': {
            'immunomodulator': 0.7,
            'antioxidant': 0.8,
            'adaptogenic': 0.6,
            'nutritive': 0.8,
            'high_sugar_content': 0.9
        },
        'safety_concerns': ['high sugar content', 'hypoglycemia risk with diabetes meds', 'allergy potential'],
        'contraindicated_conditions': ['uncontrolled diabetes', 'allergy to ingredients'],
        'organ_tropism': ['immune_system', 'respiratory_system', 'general']
    }
]

ayurvedic_profiles['herbs'] = herb_profiles
ayurvedic_profiles['metadata'] = {
    'version': '1.0',
    'date_created': datetime.now().strftime('%Y-%m-%d'),
    'total_herbs': len(herb_profiles),
    'profile_scoring_scale': '0.0 (no effect) to 1.0 (strong effect)',
    'data_sources': [
        'Traditional Ayurvedic texts (Charaka Samhita, Sushruta Samhita)',
        'Bhavaprakasha Samhita',
        'Modern pharmacological research',
        'Clinical studies',
        'Dravyaguna Vijñāna (Ayurvedic pharmacology)',
        'IMPPAT Database',
        'Peer-reviewed publications'
    ],
    'note': 'Pharmacological profiles are approximate classifications for decision support. Clinical validation required.'
}

# Save as JSON
with open('ayurvedic_pharmacological_profiles.json', 'w', encoding='utf-8') as f:
    json.dump(ayurvedic_profiles, f, indent=2, ensure_ascii=False)

print(f"✓ Created ayurvedic_pharmacological_profiles.json with {len(herb_profiles)} herbs")

# Create summary CSV for quick reference
summary_data = []
for herb in herb_profiles:
    summary_data.append({
        'Herb Name': herb['herb_name'],
        'Scientific Name': herb['scientific_name'],
        'Primary Indications': ', '.join(herb['common_indications'][:3]),
        'Key Safety Concerns': ', '.join(herb['safety_concerns']),
        'Organ Systems': ', '.join(herb['organ_tropism'])
    })

df_summary = pd.DataFrame(summary_data)
df_summary.to_csv('ayurvedic_herbs_summary.csv', index=False)
print(f"✓ Created ayurvedic_herbs_summary.csv with {len(df_summary)} herbs")

print("\n" + "="*80)
print("DATASET CREATION COMPLETE")
print("="*80)
print(f"\nFiles created:")
print(f"1. ayurvedic_known_interactions.csv - {len(df_known)} documented interactions")
print(f"2. ayurvedic_pharmacological_profiles.json - {len(herb_profiles)} herb profiles")
print(f"3. ayurvedic_herbs_summary.csv - Quick reference summary")
print("\nAll data sourced from validated literature and traditional Ayurvedic texts.")


✓ Created ayurvedic_known_interactions.csv with 38 interactions
✓ Created ayurvedic_pharmacological_profiles.json with 28 herbs
✓ Created ayurvedic_herbs_summary.csv with 28 herbs

DATASET CREATION COMPLETE

Files created:
1. ayurvedic_known_interactions.csv - 38 documented interactions
2. ayurvedic_pharmacological_profiles.json - 28 herb profiles
3. ayurvedic_herbs_summary.csv - Quick reference summary

All data sourced from validated literature and traditional Ayurvedic texts.


In [28]:
import pandas as pd
import re

# ==========================================
# CONFIGURATION
# ==========================================
INPUT_FILE = "acb_score.csv" # Rename this to your actual file
OUTPUT_FILE = "acb_scores_final.csv"

def clean_brand_names(text):
    if pd.isna(text) or text == "":
        return ""
    # Remove replacement characters and trademark symbols
    text = str(text).replace("", "").replace("™", "")
    # Remove extra commas or whitespace
    return re.sub(r'\s+,', ',', text).strip(" ,")

def validate_scores(df):
    # Ensure scores are 0, 1, 2, or 3. 
    # If a score is missing or weird, default to 0 (Safety default)
    df["Score"] = pd.to_numeric(df["Score"], errors='coerce').fillna(0).astype(int)
    return df

def main():
    try:
        # Read with 'latin1' or 'cp1252' if utf-8 fails due to those symbols
        df = pd.read_csv(INPUT_FILE, encoding_errors='replace')
        
        # 1. Clean the Brand Name Column
        if "Brand" in df.columns:
            df["Brand"] = df["Brand"].apply(clean_brand_names)
            
        # 2. Validate Scores
        df = validate_scores(df)
        
        # 3. Add Clinical Context (Useful for your App)
        # We add a column explaining what the score MEANS.
        def get_risk(score):
            if score >= 3: return "High Risk (Severe)"
            if score == 2: return "Moderate Risk"
            if score == 1: return "Mild Risk"
            return "No Known Risk"
            
        df["Risk_Category"] = df["Score"].apply(get_risk)

        # 4. Save clean version
        df.to_csv(OUTPUT_FILE, index=False, encoding='utf-8-sig')
        print(f"Success! Cleaned data saved to {OUTPUT_FILE}")
        print(df.head())
        
    except Exception as e:
        print(f"Error: {e}")
        print("Make sure the input filename matches your saved CSV.")

if __name__ == "__main__":
    main()

Success! Cleaned data saved to acb_scores_final.csv
                  Name Brand  Score  Risk_Category
0  5-hydroxytryptophan            0  No Known Risk
1             Abacavir            0  No Known Risk
2          Abiraterone            0  No Known Risk
3          Acamprosate            0  No Known Risk
4             Acarbose            0  No Known Risk


In [29]:
import csv

# Data extracted from the image
records = [
    ("Alimemazine", "Theralen™"),
    ("Alverine", "Spasmonal™"),
    ("Alprazolam", "Xanax™"),
    ("Aripiprazole", "Abilify™"),
    ("Asenapine", "Saphris™"),
    ("Atenolol", "Tenormin™"),
    ("Bupropion", "Wellbutrin™, Zyban™"),
    ("Captopril", "Capoten™"),
    ("Cetirizine", "Zyrtec™"),
    ("Chlorthalidone", "Diuril™, Hygroton™"),
    ("Cimetidine", "Tagamet™"),
    ("Clidinium", "Librax™"),
    ("Clorazepate", "Tranxene™"),
    ("Codeine", "Contin™"),
    ("Colchicine", "Colcrys™"),
    ("Desloratadine", "Clarinex™"),
    ("Diazepam", "Valium™"),
    ("Digoxin", "Lanoxin™"),
    ("Dipyridamole", "Persantine™"),
    ("Disopyramide", "Norpace™"),
    ("Fentanyl", "Duragesic™, Actiq™"),
    ("Furosemide", "Lasix™"),
    ("Fluvoxamine", "Luvox™"),
    ("Haloperidol", "Haldol™"),
    ("Hydralazine", "Apresoline™"),
    ("Hydrocortisone", "Cortef™, Cortaïd™"),
    ("Iloperidone", "Fanapt™"),
    ("Isosorbide", "Isordil™, Ismo™"),
    ("Levocetirizine", "Xyzal™"),
    ("Loperamide", "Immodium™, others"),
    ("Loratadine", "Claritin™"),
    ("Metoprolol", "Lopressor™, Toprol™"),
    ("Morphine", "MS Contin™, Avinza™"),
    ("Nifedipine", "Procardia™, Adalat™"),
]

# Save to CSV
output_file = "acb_brand_list.csv"

with open(output_file, "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["Generic Name", "Brand Name"])  # header
    writer.writerows(records)

print(f"Saved {len(records)} entries to {output_file}")


Saved 34 entries to acb_brand_list.csv


In [30]:
import csv

# ---------- Existing entries (previous screenshot) ----------
records = [
    ("Alimemazine", "Theralen™", 1),
    ("Alverine", "Spasmonal™", 1),
    ("Alprazolam", "Xanax™", 1),
    ("Aripiprazole", "Abilify™", 1),
    ("Asenapine", "Saphris™", 1),
    ("Atenolol", "Tenormin™", 1),
    ("Bupropion", "Wellbutrin™, Zyban™", 1),
    ("Captopril", "Capoten™", 1),
    ("Cetirizine", "Zyrtec™", 1),
    ("Chlorthalidone", "Diuril™, Hygroton™", 1),
    ("Cimetidine", "Tagamet™", 1),
    ("Clidinium", "Librax™", 1),
    ("Clorazepate", "Tranxene™", 1),
    ("Codeine", "Contin™", 1),
    ("Colchicine", "Colcrys™", 1),
    ("Desloratadine", "Clarinex™", 1),
    ("Diazepam", "Valium™", 1),
    ("Digoxin", "Lanoxin™", 1),
    ("Dipyridamole", "Persantine™", 1),
    ("Disopyramide", "Norpace™", 1),
    ("Fentanyl", "Duragesic™, Actiq™", 1),
    ("Furosemide", "Lasix™", 1),
    ("Fluvoxamine", "Luvox™", 1),
    ("Haloperidol", "Haldol™", 1),
    ("Hydralazine", "Apresoline™", 1),
    ("Hydrocortisone", "Cortef™, Cortaïd™", 1),
    ("Iloperidone", "Fanapt™", 1),
    ("Isosorbide", "Isordil™, Ismo™", 1),
    ("Levocetirizine", "Xyzal™", 1),
    ("Loperamide", "Immodium™, others", 1),
    ("Loratadine", "Claritin™", 1),
    ("Metoprolol", "Lopressor™, Toprol™", 1),
    ("Morphine", "MS Contin™, Avinza™", 1),
    ("Nifedipine", "Procardia™, Adalat™", 1),
]

# ---------- NEW entries (from latest screenshot) ----------
new_records = [
    ("Paliperidone", "Invega™", 1),
    ("Prednisone", "Deltasone™, Sterapred™", 1),
    ("Quinidine", "Quinaglute™", 1),
    ("Ranitidine", "Zantac™", 1),
    ("Risperidone", "Risperdal™", 1),
    ("Theophylline", "Theodur™, Uniphyl™", 1),
    ("Trazodone", "Desyrel™", 1),
    ("Triamterene", "Dyrenium™", 1),
    ("Venlafaxine", "Effexor™", 1),
    ("Warfarin", "Coumadin™", 1),
]

# Add to main list
records.extend(new_records)

# ---------- Save CSV ----------
output_file = "acb_brand_list.csv"

with open(output_file, "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["Generic Name", "Brand Name", "ACB Score"])
    writer.writerows(records)

print(f"Saved {len(records)} medicines to {output_file}")


Saved 44 medicines to acb_brand_list.csv


In [32]:
import csv

# New ACB = 2 entries from the screenshot
acb2_records = [
    ("Amantadine", "Symmetrel™", 2),
    ("Belladonna", "Multiple", 2),
    ("Carbamazepine", "Tegretol™", 2),
    ("Cyclobenzaprine", "Flexeril™", 2),
    ("Cyproheptadine", "Periactin™", 2),
    ("Loxapine", "Loxitane™", 2),
    ("Meperidine", "Demerol™", 2),
    ("Methotrimeprazine", "Levoprome™", 2),
    ("Molindone", "Moban™", 2),
    ("Nefopam", "Nefogesic™", 2),
    ("Oxcarbazepine", "Trileptal™", 2),
    ("Pimozide", "Orap™", 2),
]

csv_file = "acb_brand_list.csv"

# Append mode: add new rows without touching existing ones
with open(csv_file, "a", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerows(acb2_records)

print(f"Added {len(acb2_records)} ACB=2 medicines to {csv_file}")


Added 12 ACB=2 medicines to acb_brand_list.csv


In [34]:
import csv

# ACB = 3 medicines from your screenshot
acb3_records = [
    ("Trifluoperazine", "Stelazine™", 3),
    ("Trihexyphenidyl", "Artane™", 3),
    ("Trimipramine", "Surmontil™", 3),
    ("Trospium", "Sanctura™", 3),
]

csv_file = "acb_brand_list.csv"

# Append to the existing CSV file
with open(csv_file, "a", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerows(acb3_records)

print(f"Added {len(acb3_records)} ACB=3 medicines to {csv_file}")


Added 4 ACB=3 medicines to acb_brand_list.csv
